In [7]:
import torch.nn as nn
import torch
from torchvision import datasets,transforms
from torch.utils.data import DataLoader,Dataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import matplotlib.pyplot as plt

In [8]:
z_dim = 100
img_size = 64
batch_size = 64

In [9]:
class Generator(nn.Module):
    def __init__(self,z_dim,img_size):
        super(Generator,self).__init__()
        self.img_size = img_size
        self.z_dim = z_dim
        self.net = nn.Sequential(
            nn.Linear(self.z_dim,256),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(256),

            nn.Linear(256,512),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(512),

            nn.Linear(512,1024),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(1024),

            nn.Linear(1024,self.img_size**2),
        )

    def forward(self, x):
        return self.net(x)

In [10]:
class Discriminator(nn.Module):
    def __init__(self,img_size):
        super(Discriminator,self).__init__()
        self.net = nn.Sequential(
            nn.Linear(img_size,1024),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(1024),

            nn.Linear(1024,512),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(512),

            nn.Linear(512,256),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(256),

            nn.Linear(256,1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

In [11]:
discriminator = Discriminator(img_size)
generator = Generator(z_dim,img_size)

In [12]:
criterion = nn.BCELoss()
optimizer_G = torch.optim.Adam(generator.parameters(),lr=0.0002)
optimizer_D = torch.optim.Adam(discriminator.parameters(),lr=0.0002)

In [ ]:
def train(num_epochs,train_loader,d_optimizer,g_optimizer):
    generator.train()
    discriminator.train()
    epochs = torch.arange(1, num_epochs+1)
    g_losses = []
    d_losses = []

    for epoch in range(num_epochs):
        g_running_loss = 0.0
        d_running_loss = 0.0
        for i, (real_images,_) in enumerate(train_loader):  # 遍历数据集，每次获取一个 batch 的真实图像
            d_optimizer.zero_grad()  # 清空判别器的梯度

            real_images = real_images.to(device)  # 将真实图像移动到计算设备
            real_outputs = discriminator(real_images)  # 判别器对真实图像的预测
            real_labels = torch.ones(batch_size).to(device)  # 真实样本的标签设为 1

            z = torch.randn(batch_size, z_dim, 1 , 1).to(device)  # 生成 batch_size 个随机噪声向量
            fake_images = generator(z)  # 生成器生成假的图像
            fake_outputs = discriminator(fake_images.detach())  # 判别器对假图像的预测（detach 避免更新生成器）
            fake_labels = torch.zeros(batch_size).to(device)  # 假样本的标签设为 0

            d_real_loss = criterion(real_outputs, real_labels)  # 计算判别器在真实图像上的损失
            d_fake_loss = criterion(fake_outputs, fake_labels)  # 计算判别器在假图像上的损失
            d_loss = d_real_loss + d_fake_loss  # 判别器的总损失

            d_loss.backward()  # 反向传播更新判别器
            d_optimizer.step()  # 进行梯度更新

            g_optimizer.zero_grad()  # 清空生成器的梯度
            g_outputs = discriminator(fake_images)  # 让判别器评估生成的假图像
            g_loss = criterion(g_outputs, real_labels)  # 计算生成器的损失（希望判别器认为假图像是真实的）
            g_loss.backward()  # 反向传播更新生成器
            g_optimizer.step()  # 进行梯度更新

            g_running_loss += g_loss.item()  # 记录生成器损失
            d_running_loss += d_loss.item()  # 记录判别器损失

            if (i+1) % 100 == 0:
                print(f"Epoch : [{epoch+1}/{num_epochs}]\tIter : [{i+1}/{len(train_loader)}]\tGenerator Loss: {g_running_loss/(i+1):.3f}\tDiscriminator Loss: {d_running_loss/(i+1):.3f} ",'\n')

        g_losses.append(g_running_loss/len(train_loader))
        d_losses.append(d_running_loss/len(train_loader))

    fig, axes = plt.subplots(1, 2, figsize=(10, 4))  # 1行2列的子图

    # 绘制 Generator Loss
    axes[0].plot(epochs, g_losses, marker='o', color='blue', label='Generator Loss')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('G Loss')
    axes[0].set_title('Generator Loss Curve')
    axes[0].legend()

    # 绘制 Discriminator Loss
    axes[1].plot(epochs, d_losses, marker='s', color='red', label='Discriminator Loss')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('D Loss')
    axes[1].set_title('Discriminator Loss Curve')
    axes[1].legend()

    plt.tight_layout()
    plt.show()

In [ ]:
from torchvision.utils import make_grid


def show_generated_images(generator, z_dim, device, num_images=64):
    # 生成随机噪声
    noise = torch.randn(num_images, z_dim, 1, 1, device=device)
    # 使用生成器生成假图像，并确保不计算梯度
    with torch.no_grad():
        fake_images = generator(noise).detach().cpu()

    # 利用 torchvision 的 make_grid 函数将多张图片拼接成一个网格
    image_grid = make_grid(fake_images, padding=2, normalize=True)

    # 将图像转换成 numpy 数组，并转换通道顺序以适应 matplotlib (H, W, C)
    np_image = image_grid.numpy().transpose((1, 2, 0))

    # 使用 matplotlib 显示图像
    plt.figure(figsize=(8, 8))
    plt.imshow(np_image)
    plt.axis("off")
    plt.show()
